# Shor algorithm

This notebook is part of a set of notebooks on different quantum algorithms. The main source of these notes is: (Vedral, 2013). 

## Theory

### Shor's algorithm

Shor's algorithm is a method for factoring a natural number $N$. It relies on the existance of an efficient classical algorithm for finding the greatest common factor (gcf) (called Euclid's algorithm (Lynn, 2000)) and on an efficient method to deduce some random eigenstates of an operator (called quantum phase estimation).

The algorithm is (based on (Vedral, 2013, p. 140)):

1. Randomly generate some natural number $a$ where $2<a<N$.

2. If $a$ and $N$ are not coprime $\left(K = gcf(a,N) \neq 1\right)$, then the factorization of $N$ is $\frac{N}{K},K$ and the algorithm stopped.

3. If $a$ is odd, then the algorithm fails. Otherwise, deduce the order of $a$ (smallest $r$ where $a^r \bmod N = 1$) by the phase estimation method (see below).

4. Then, as $a^r -1 \bmod N = (a^\frac{r}{2} - 1)(a^\frac{r}{2} + 1) \bmod N  = 0$ and as $a^\frac{r}{2} \bmod N  = 1$ can't be true (by the definition of $r$), if $N$ doesn't divide $b = a^\frac{r}{2} + 1$ then $a$ and $b$ share common factors. These can be deduced by Euclid's algorithm. Meaning if $K = gcd(a,b) \neq 1$ then the factorization of $N$ is $\frac{N}{K},K$.

It can be shown that this process has a success probability of $\frac{1}{2}$ (Vedral, 2013, p. 142) and the complexity of the algorithm can be shown to be $O((\log{n})^2(\log{\log{n}})(\log{\log{\log{n}}}))$ (Shor, 1996, p. 15).

### Euclid's algorithm

Euclid's algorithm finds the gcf of two natural numbers (here these numbers shall be denoted by $a,b$ and it will be assumed that $a<b$). The algorithm works by recursively applying the formula $gcd(a,b) = gcd(a,b-a)$ until the gcd is trivial (Lynn, 2000).

### Phase estimation

This is the quantum part of the algorithm; it efficiently finds the order $r$ associated with $a$ and $N$. To find $r$, a qubit system of dimension $m >\log_2(N)$ is set up with the normal binary labelling convention used for the basis (eg. $\ket{0} = \ket{000}, \ket{1} = \ket{010}, \ket{2} = \ket{011},...$). An operator $\hat{U}_a$ is set up where $\hat{U}_a\ket{x} = \ket{ax \bmod N}$ (for efficient implementation see (Shor, 1996, p. 11)). It can then be shown that the eigenstates of $\hat{U}_a$ take the form $\lambda_n = e^{\frac{2\pi i n}{r}}$ and that $\ket{1}$ is proportional to the sum of all eigenkets (Vendral, 2013, p. 144). Due to these properties, the quantum phase estimation algorithm can then be applied to $\ket{1}$ (see (Shor, 1996, p. 15) and (Vendral, 2013, p. 142) for a description and circuit diagrams). The result is an equally weighted superposition corresponding to each of the different eigenstates. A measurement can then be made to find one of these eigenstates. The resulting number is approximately $2^{2m+1}\frac{n}{r}$. $r$ can then be approximated by the denominator of the coprime fraction closest to $\frac{n}{r}$. This approximation can be made more accurate by performing the algorithm several times and taking the lowest common multiple (LCM) of the resulting r values.

## Implmentation

Below the algorithm is implemented. For simplicity, the implementation only considers product states and the quantum randomness is simulated by randomly selecting an eigenstate for phase estimation to be applied to. Test cases were taken from: https://blendmaster.github.io/ShorJS/.



In [14]:
import numpy as np
import itertools

# basis used is |0> = [1,0], |1> = [0,1]
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

Code for inverse QFT.

In [15]:
def invQFT(f):
    """
    Performs inverse quantum fourier transform on a tensor products of qubit states.

    Intputs:
    f - tensor product of qubit states, in form [[qubit 1 state], [qubit 2 state], ...]
    """

    qNum = f.shape[0]
    fInv = np.empty([qNum,2],dtype=np.complex_)
    fInv[-1] = H @ f[-1]

    for i in reversed(range(qNum-1)): 
        s = f[i]
        for k in range(i+1,qNum):
            if np.allclose(fInv[k],np.array([0,1])):
                s[1] *= np.exp(-2*np.pi*complex(0,1)/(2**(k+1)))
        s = H @ s
        fInv[i] = s
    
    return fInv


In [16]:
def getDecimalLabel(s): 
    """
    Function that finds the associated number for a qubit tensor product state. 
    """
    #print(s)
    sBinary = np.empty(s.shape[0])
    for i,ket in enumerate(s):
        if np.allclose(ket,np.array([1,0])):
            sBinary[i] = 0
        elif np.allclose(ket,np.array([0,1])):
            sBinary[i] = 1
    
    number = sum([b*2**i for i,b in enumerate(np.flip(sBinary))])
    return number


Test cases for inverse QFT.

In [17]:
#result should be 2
f1 = np.array([[1,complex(0,1)],[1,-1],[1,1]])/np.sqrt(2) 
print(f"Result: {getDecimalLabel(invQFT(f1))}")

Result: 2.0


In [18]:
#result should be 1
f2 = [np.array([1,complex(0,1)]),np.array([1,-1])]/np.sqrt(2)
print(f"Result: {getDecimalLabel(invQFT(f2))}")

Result: 1.0


Code for phase estimation.

In [19]:

def phaseEstimation(psiAcc,U,e): #reversed to tensor product convention used in QFT
    """
    Performs phase estimation.

    Inputs:
    psiAcc - accuracy of psi returned
    U - unitary matrix
    e - normalised ket
    """
    N = U.shape[0]

    def pow(n,prev=np.identity(N,dtype=np.complex_)):
        if n == 0:
            return prev
        return pow(n-1,U @ prev)
    

    f = np.zeros([psiAcc,2],dtype=np.complex_)

    for i in range(psiAcc):
        s = np.array([1,0],dtype=np.complex_)
        s = H @ s

        #apply U^2^i
        s[1] *= e @ pow(2**i) @ e
        
        f[i] = s
    
    finvQFT = invQFT(f)
    phase = getDecimalLabel(finvQFT)/2**psiAcc #from relating the phase and result of inv QFT 

    return phase


Tests for phase estimation function.

In [20]:
#result should be 0
U1 = np.array([[1,0],[0,1]])
e1 = np.array([1,0],dtype=np.complex_)
print(f"Psi: {phaseEstimation(9,U1,e1)}")

Psi: 0.0


In [21]:
#result should be 0.25
U2 = np.array([[complex(0,1),0],[0,1]])
e2 = np.array([1,0],dtype=np.complex_)
print(f"Psi: {phaseEstimation(9,U2,e2)}")

Psi: 0.25


In [22]:
#result shoudl be 0.5 
U3 = np.array([[np.exp(np.pi*complex(0,1)),0],[0,1]],dtype=np.complex_) 
e3 = np.array([1,0],dtype=np.complex_)
print(f"Psi: {phaseEstimation(8,U3,e3)}")

Psi: 0.5


Code for full Shor algorithm.

In [23]:
def findNearestRational(x):
    """
    Function that finds the two coprime numbers whose fraction is nearest to x.
    """
    xb = np.round(x)
    xa = x - xb
    if xa != 0:
        xaRecpb = np.round(1/xa)
        return xb*xaRecpb+1,xaRecpb
    else:
        return xb,1
      

def shor(N,a,qNum,m,psiAcc):
    """
    Function that factorizes natural number N.

    Inputs:
    N - number being factorized
    a - int between 2 and N used in algorithm
    qNum - number of qubits used for U_a matrix
    m - number of runs of phase estimation algorithm
    psiAcc - accuracy of psi found in phase estimation
    """
    HDim = 2**qNum

    K = np.gcd(a,N)
    if K != 1 and K != N:
        return N/K,K
    
    #find U_a matrix
    U_a = np.zeros([HDim,HDim],dtype=np.complex_)
    for j,k in itertools.product(range(HDim), repeat=2): 
        if j == a*k % N and k < N:
            U_a[j,k] = 1
        elif j == k and k >= N:
            U_a[j,k] = 1
            
    #random number generator models quantum probablities
    eigenVal, eigenVec = np.linalg.eig(U_a)
    
    rs = []
    for l in range(m):
        index = np.random.randint(0,high=HDim)
        phaseEst = phaseEstimation(psiAcc,U_a,eigenVec[index])
        if phaseEst != None:
            kj,rj = findNearestRational(phaseEst)
            rs.append(rj)
    
    if rs != []:
        R = np.lcm.reduce(np.array(rs,dtype=int))

        if R % 2 == 0:
            g = np.gcd(int(a**(R/2) + 1),int(N)) 
            if g != 1 and g != N:
                return N/g, g
    
    return "failed"


Tests for Shor algorithm.

In [24]:
print(f"12 is factorized to: {shor(12,5,4,6,4)}") 

12 is factorized to: (6.0, 2)


In [25]:
#should fail
print(f"21 is factorized to: {shor(21,5,4,6,4)}") 

21 is factorized to: failed


# References

Vedral, V., 2013. Introduction to Quantum Information Science. Oxford: Oxford University Press.

Lynn, B., 2000. Euclid's Algorithm. [Online] 
Available at: https://crypto.stanford.edu/pbc/notes/numbertheory/euclid.html
[Accessed 27 August 2024].

Shor, P., 1996. Polynomial-Time Algorithms for Prime Factorization and Discrete Logarithms on a Quantum Computer. 



